In [61]:
from rdflib import Graph
from pydantic import BaseModel, validator, Field
from typing import Optional, Union
from pyfuseki import FusekiUpdate
import json
from datetime import date, datetime
from api.src.function.bibframe.Work.graph import MakeGraph
import httpx
from api.src.schemas.bibframe.work import Work
import pysolr  

In [19]:
g = Graph()
g.parse('http://id.loc.gov/resources/works/21291853.nt', format='nt')
g.serialize('loc1.ttl', format='ttl')

<Graph identifier=Na57cfb0002414db99f53ba7608959db3 (<class 'rdflib.graph.Graph'>)>

In [18]:
g = Graph()
g.parse('preview.rdf')
g.serialize('preview.ttl', format='ttl')

<Graph identifier=N4365006d7cb747c8ac8ac0b76cd0b878 (<class 'rdflib.graph.Graph'>)>

In [2]:
with open('api/src/config/authorized_values.json') as file:
    reader = file.read()
    vocabulary = json.loads(reader)
    file.close()

In [48]:
with open('work.json') as file:
    reader = file.read()
    work = json.loads(reader)
    file.close()
request = Work(**work)

In [49]:
request.contribution

[Contribution(type=['bf:Contribution', 'bflc:PrimaryContribution'], agent='https://bibliokeia.com/authorities/name/n80002329', label='Machado de Assis', role='relators:aut')]

In [16]:
request = Work(**work)
graph = MakeGraph(request, 'bk-1')
print(graph)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/> 
    PREFIX bkw: <https://bibliokeia.com/resources/work/> 
    PREFIX menclvl: <http://id.loc.gov/vocabulary/menclvl/>
    PREFIX mstatus: <http://id.loc.gov/vocabulary/mstatus/>
    PREFIX contentTypes: <http://id.loc.gov/vocabulary/contentTypes/>
    PREFIX relators: <http://id.loc.gov/vocabulary/relators/>
    PREFIX genreForms: <http://id.loc.gov/authorities/genreForms/>
    PREFIX msupplcont: <http://id.loc.gov/vocabulary/msupplcont/>
    PREFIX millus: <http://id.loc.gov/vocabulary/millus/>    
        
    INSERT DATA {
        GRAPH bkw:bk-1
        { 
           bkw:bk-1 a bf:Monograph, bf:Text, bf:Work  ;
           bf:adminMetadata [ a bf:AdminMetada

 ns1:assigner <http://id.loc.gov/vocabulary/organizations/dlc> ;
            ns1:changeDate "2023-02-06T08:03:19"^^xsd:dateTime ;
            ns1:creationDate "2019-11-26"^^xsd:date ;
            ns1:descriptionAuthentication <http://id.loc.gov/vocabulary/marcauthen/pcc> ;
            ns1:descriptionConventions <http://id.loc.gov/vocabulary/descriptionConventions/isbd>,
                <http://id.loc.gov/vocabulary/descriptionConventions/rda> ;
            ns1:descriptionLanguage <http://id.loc.gov/vocabulary/languages/eng> ;
            ns1:descriptionModifier <http://id.loc.gov/vocabulary/organizations/dlc> ;
            ns1:generationProcess [ a ns1:GenerationProcess ;
                    rdfs:label "DLC marc2bibframe2 v2.2.1" ;
                    ns1:generationDate "2023-02-07T01:58:10.983465-05:00"^^xsd:dateTime ] ;
            ns1:identifiedBy [ a ns1:Local ;
                    ns1:assigner <http://id.loc.gov/vocabulary/organizations/dlc> ;
                    rdf:value "21313495" ] ;
            ns1:status <http://id.loc.gov/vocabulary/mstatus/c> ;

In [17]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'acervo')

response = fuseki_update.run_sparql(graph)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [18]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'acervo')
d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

response = fuseki_update.run_sparql(d)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

Exemple Work Loc

In [43]:
g = Graph()
g.parse('http://id.loc.gov/authorities/names/n80002329.rdf')

<Graph identifier=Nc98b5d06e8534accb4416bdc19980ca7 (<class 'rdflib.graph.Graph'>)>

In [44]:
g.serialize('name.ttl', format='ttl')

<Graph identifier=Nc98b5d06e8534accb4416bdc19980ca7 (<class 'rdflib.graph.Graph'>)>

GET WORK

In [18]:
response = httpx.get('http://localhost:3030/acervo?graph=https://bibliokeia.com/resources/work/bk-1')
response

<Response [200 OK]>

In [19]:
g = Graph()
g.parse(response.content)
g.serialize('bk.ttl', format='ttl')

<Graph identifier=Naae5efcb5c2b4873905cc6b83fb657ac (<class 'rdflib.graph.Graph'>)>

In [51]:
for contribution in request.contribution:
    print(contribution.agent)

https://bibliokeia.com/authorities/name/n80002329


In [42]:
fuseki_authorities = FusekiUpdate('http://localhost:3030', 'authorities')
subjectOf = """PREFIX graph: <https://bibliokeia.com/authorities/topic/>
            PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>

                INSERT DATA
                    {{ GRAPH <{subject}> {{ 
                        <{subject}>  bf:contributionOf   <https://bibliokeia.com/resources/work/{id}> }} }} ; """
for i in request.subject:
    up = subjectOf.format(subject=i.value, id="bk-1")
    response = fuseki_authorities.run_sparql(up)
    print(response.convert())

{'statusCode': 200, 'message': 'Update succeeded'}


In [32]:
fuseki_authorities = FusekiUpdate('http://localhost:3030', 'authorities')
subjectOf = f"""PREFIX graph: <https://bibliokeia.com/authorities/topic/>
            PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>

                INSERT DATA
                    {{ GRAPH graph:sh85017405 {{ 
                        graph:sh85017405  bf:subjectOf <https://bibliokeia.com/resources/work/bk-2> }} }} ; """
fuseki_authorities.run_sparql(subjectOf)

In [60]:
request.dict()
id = 'bk-1'
doc = {
    'id': id,
    'type': [i.replace("bf:", "") for i in request.type],
    'adminMetadata': request.adminMetadata.dict()
}
doc

{'id': 'bk-1',
 'type': ['Monograph', 'Text', 'Work'],
 'adminMetadata': {'encodingLevel': 'menclvl:1',
  'assigner': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'creationDate': '2023-04-19',
  'descriptionConventions': 'http://id.loc.gov/vocabulary/descriptionConventions/isbd',
  'descriptionLanguage': 'http://id.loc.gov/vocabulary/languages/por',
  'generationProcess': {'label': 'BiblioKeia',
   'generationDate': '2023-04-19T17:11:33'},
  'status': 'mstatus:new'}}

In [62]:
solr = pysolr.Solr('http://localhost:8983/solr/acervo/', timeout=10)

In [63]:
r = solr.add([doc], commit=True)
r

SolrError: Solr responded with an error (HTTP 400): [Reason: ERROR: [doc=bk-1/adminMetadata#/generationProcess#] unknown field 'generationDate']